- Title: Classificando Nomes por Gênero Usando Dados Públicos
- Slug: classificando-nomes-por-genero-usando-dados-publicos
- Date: 2019-05-31 15:10
- Category: datasets
- Tags: python, data science
- Author: turicas
- Summary: Muitos datasets que possuem dados sobre pessoas não possuem a informação de gênero correspondente, dificultando projetos de dados onde o recorte por gênero é necessário. Nesse artigo vamos aprender a utilizar um dataset do Brasil.IO para fazer essa classificação!

Muitos *datasets* que possuem dados sobre pessoas não possuem a informação de gênero correspondente, como o *dataset* de candidatura das eleições para o ano de 1.996 divulgado pelo [Tribunal Superior Eleitoral](http://www.tse.jus.br/) ([veja esse *dataset* no Brasil.IO](https://brasil.io/dataset/eleicoes-brasil/candidatos?ano_eleicao=1996)) e o *dataset* de salários dos magistrados divulgado pelo [Conselho Nacional de Justiça](http://cnj.jus.br/) ([veja esse *dataset* no Brasil.IO](https://brasil.io/dataset/salarios-magistrados/contracheque)). A falta dessa informação dificulta projetos de dados onde o recorte por gênero é necessário, como é o caso [desse vídeo](https://www.youtube.com/watch?v=7yQ8U2tFFq4) que ajudei a construir, onde classificamos por gênero todos os nomes de ruas, avenidas e praças do Brasil. 

Nesse artigo vamos aprender a utilizar os dados de [nomes brasileiros do Censo 2010 do IBGE](https://censo2010.ibge.gov.br/nomes/#/search) que estão [disponíveis no Brasil.IO](https://brasil.io/dataset/genero-nomes) para fazer essa classificação, utilizando como base mais de 100 mil nomes e a [linguagem Python](https://www.python.org/)! :)

## Instalando as dependências

Antes de iniciar o código criaremos um ambiente virtual (virtualenv) utilizando a versão 3.7 do Python (vou utilizar o
[pyenv](https://github.com/pyenv/pyenv) e [pyenv-virtualenv](https://github.com/pyenv/pyenv-virtualenv) para isso, mas fique à vontade para utilizar outras ferramentas em seu ambiente). Para esse projeto utilizaremos apenas as bibliotecas da [*standard library* do Python](https://docs.python.org/3/library/), ou seja, não precisaremos instalar bibliotecas externas.
 
```bash
pyenv virtualenv 3.7.3 classificador-genero
pyenv activate classificador-genero
```

## Entendendo o Dataset

A [tabela nomes do *dataset* genero-nomes](https://brasil.io/dataset/genero-nomes/nomes) possui a lista de todos os nomes encontrados no Censo 2010, contendo a frequência desse nome para os gêneros masculino e feminino, a classificação baseada nessa frequência (o gênero que tiver frequência maior) e a probabilidade da classificação ser verdadeira (frequência do gênero que aparece mais dividida pela frequência total do nome), como podemos ver na seguinte tabela:

![Tabela de gênero dos nomes]({static}/images/2019-05-31-tabela-genero-nomes.png)

Para o nome ABADIAS, por exemplo, existem 45 pessoas do gênero feminimo e 156 do masculino, então o nome é classificado com o gênero `M` (masculino) com uma probabilidade de 78% (`156 / (156 + 45)`).

Repare também que os nomes contidos nesse *dataset* estão todos em letras maiúsculas e sem acentos, então precisaremos criar uma função para transformar o nome que queremos classificar (exemplo: `Álvaro`) para esse formato. Para isso, vamos utilizar a função `normalize` do módulo [unicodedata](https://docs.python.org/3/library/unicodedata.html) para separar os acentos das letras e ignorar os erros ao converter para o *encoding* ASCII, que eliminará todos os símbolos que não queremos:

In [1]:
from unicodedata import normalize

def encode(name):
    ascii_name = normalize("NFKD", name).encode("ascii", errors="ignore").decode("ascii")
    return ascii_name.upper()

print(encode("Álvaro"))

ALVARO


## Possíveis Abordagens

Para esse projeto vamos fazer um **classificador simples**, sem a utilização de técnicas de aprendizado de máquina, ou seja, ele será capaz de classificar apenas os nomes já disponíveis em nosso dataset. Em próximos artigos mostraremos como criar outros tipos de classificadores.

Para resolver essa questão, temos duas abordagens possíveis:
- Utilizar a [API do Brasil.IO](https://brasil.io/api/datasets), fazendo uma requisição a cada nome a ser classificado; e
- Baixar o dataset completo e utilizar os dados localmente.

Cada uma das abordagens tem vantagens e desvantagens, que veremos a seguir.

## Método 1: Utilizando a API

Vantagens:

- Não precisaremos baixar o dataset completo
- Utiliza pouca memória

Desvantagens:

- Classificação lenta (uma requisição HTTP por classificação)

Vamos criar uma função que recebe o nome, faz a requisição HTTP usando a [urllib](https://docs.python.org/3/library/urllib.html), lê os dados com a [json](https://docs.python.org/3/library/json.html) e retorna o resultado encontrado:

In [2]:
import json
from urllib.request import urlopen

def classify_api(name):
    encoded_name = encode(name)
    url = "https://brasil.io/api/dataset/genero-nomes/nomes/data?first_name=" + encoded_name
    response = urlopen(url)
    json_response = json.loads(response.read())
    return json_response["results"][0]["classification"]

print(classify_api("Álvaro"))

M


Fica como sugestão de exercício melhorar a função acima, adicionando tratamento de erros caso o nome não seja encontrado.

## Método 2: Baixando os dados

Vantagens:

- Classificação rápida (acesso à chave de um dicionário)

Desvantagens:

- Precisaremos baixar o dataset completo
- Utiliza mais memória

Vamos utilizar o `wget` no terminal para baixar a tabela `nomes` completa:

```bash
wget https://data.brasil.io/dataset/genero-nomes/nomes.csv.gz
```

Agora, vamos ler o arquivo CSV e criar um dicionário em que cada chave será um nome e o valor será a classificação correspondente, para isso utilizaremos os módulos [gzip](https://docs.python.org/3/library/gzip.html) (para descompactar o arquivo), [io](https://docs.python.org/3/library/io.html) (para decodificar o conteúdo do arquivo descompactado) e [csv](https://docs.python.org/3/library/csv.html) (para ler o arquivo).

In [3]:
import csv
import gzip
import io


def load_data():
    fobj = io.TextIOWrapper(gzip.open("nomes.csv.gz"), encoding="utf-8")
    csv_reader = csv.DictReader(fobj)
    data = {
        row["first_name"]: row["classification"]
        for row in csv_reader
    }
    fobj.close()
    return data

name_data = load_data()

print(f"Dicionário criado com {len(name_data)} nomes.")

Dicionário criado com 100787 nomes.


Com a variável `name_data` definida podemos criar a função de classificação facilmente:

In [4]:
def classify_download(name):
    encoded_name = encode(name)
    return name_data[encoded_name]

print(classify_download("Álvaro"))

M


## Comparando as Soluções

A primeira solução não exige tempo de *download* inicial, nem de carragamento dos dados, porém é bem mais lenta - cada classificação demora em torno de 1 segundo (esse valor pode variar, dependendo de sua conexão à Internet e à carga nos servidores do Brasil.IO):

In [5]:
%timeit classify_api("Álvaro")

1.14 s ± 309 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Já a segunda solução requer o *download* (uma única vez) de um arquivo de 1,9MB e mais um tempo inicial de quase meio segundo para carregar os dados em memória, porém é 1 milhão de vezes mais rápida - cada classificação demora em torno de 1 microssegundo (esse valor pode variar dependendo do processador e velocidades do disco e da memória):

In [6]:
%timeit load_data()
%timeit classify_download("Álvaro")

478 ms ± 37.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.02 µs ± 33.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


## Conclusão

Podemos utilizar dados públicos para enriquecer diversos outros *datasets*, possibilitando análises que antes não seriam possíveis. A utilização da API do Brasil.IO é bem simples e pode ser útil em casos de poucas consultas, mas quando a quantidade de consultas aumenta, o ideal é baixar os dados e trabalhar com eles localmente.

Curtiu o artigo? Se você acha o trabalho que desenvolvemos importante, considere
**[fazer uma doação](https://brasil.io/doe)** ou [colaborar de outras formas](https://brasil.io/colabore). ;)